<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Vector analytics and AI functionality per database version - 17.20 and Vantage 3.0
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<hr>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Overview</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage provides a suite in-database analytic capabilities for Vector embedding and analytics with support across multiple database versions.  This notebook series reviews these capablities per database version, including:</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Database Version 17.20+ and VantageCloud Enterprise 3.0</b>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Bring-Your-Own-Model (BYOM)</b> capabilities allow users to generate vector embeddings using open-source models serialized as ONNX format</li>
    <li><b>Vector data</b> stored as FLOAT columns in normal database tables</li>
    <li><b>Similarity analysis</b> using native ClearScape Analytics functions - <b>Vector Distance</b> and <b>KMeans</b></li>
    </ul>
    
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>VantageCloud Enterprise 3.1+</b>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>AI Analytic Functions</b> that leverage <b>Cloud-based LLMs</b> for text analytics, including Vector Embedding functions and RAG</li>
    <li><b>VECTOR Datatype</b> Varbyte-based array of vector data stored as single column</li>
    <li><b>Normalization</b> of vector data for efficient similarity analysis</li>
    <li><b>Similarity analysis</b> using VECTOR DATATYPE and additional functions</li>
    </ul>
    
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>VantageCloud Lake</b>
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>In-platform GPUs</b> leveraging Analytic Compute Clusters for high-scale vector embedding and other Large Language Model tasks</li>
    <li><b>Enterprise Vector Store APIs</b> for creating and managing vector data using Python and/or REST</li>
    <li><b>Similarity Search and RAG APIs</b> using Python</li>
    <li><b>Vector Store UI</b> for managing vector data</li>
    </ul>

<p style = 'font-size:28px;font-family:Arial;color:#00233C'><b>Demonstration Data</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>All of these demonstrations are based off of a small sample data set of Amazon book reviews.</p>
<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Prerequsites</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This only needs to be run once for the user environment - restart the kernel after installing the proper packages.</p>

In [ ]:
%pip install -U -r requirements.txt

<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Package Imports</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Standard practice to import required packages and libraries; execute this cell to import packages for Teradata automation as well as machine learning, analytics, utility, and data management packages.</p> 

In [ ]:
from teradataml import *
from teradatagenai import TextAnalyticsAI, TeradataAI, load_data
from teradatasqlalchemy.types import *

import getpass, os
from huggingface_hub import hf_hub_download

from IPython.display import clear_output , display as ipydisplay, Markdown
import matplotlib.pyplot as plt

# Set display options for dataframes, plots, and warnings
%matplotlib inline
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True
# Setting up BYOM install location.
configure.byom_install_location = 'td_mldb'

# use an embeddings model from the Teradata repo
model_name = 'multilingual-e5-small'
number_dimensions_output = 384
model_file_name = 'model.onnx'

# load vars json
with open('vars.json', 'r') as f:
    session_vars = json.load(f)

# Database login information
host = session_vars['environment']['host']
username = session_vars['hierarchy']['users']['business_users'][1]['username']
password = session_vars['hierarchy']['users']['business_users'][1]['password']

<hr>
<p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Features supported in database version 17.20 and Vantage Enterprise 3.0
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
    

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following demonstration will review the the process of vector embedding and similarity search using <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_Lake_VMware/Teradata-VantageTM-Bring-Your-Own-Model-User-Guide/Welcome-to-Bring-Your-Own-Model'>Bring Your Own Model (BYOM)</a> as the primary mechanism for Vector Embedding:</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '40%'>    
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Load the model</b> into the database using the teradataml Python Package</li>
    <br>
    <li><b>Generate Embeddings</b> using the ONNXEmbeddings function</li>
    <br>
    <li><b>Similarity analysis</b> using VECTORDISTANCE and KMEANS</li>
    </ol>
        </td><td style = 'text-align:center'><img src = 'images/Pattern_1.png' width = '300'></td></tr>
</table>
<hr>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Optional - Download the model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Teradata Vantage can execute various AI and ML models as functions running as native <b>MPP</b> functions inside the database. This includes models for vector embedding, text summarization, etc.  Models are loaded as ONNX (Open Neural Network eXchange) formatted files.  ONNX versions of popular models are available in the <a href = 'https://huggingface.co/Teradata'>Teradata Hugging Face repository</a>.  Other models can be converted to this format using tools such as <a href = 'https://pypi.org/project/optimum/'>optimum</a>.</p>

In [ ]:
# Optional - Download Model from Teradata HuggingFace Page
if False:
    hf_hub_download(repo_id = f'Teradata/{model_name}', filename = f'onnx/{model_file_name}', local_dir = os.getcwd())
    hf_hub_download(repo_id = f'Teradata/{model_name}', filename = 'tokenizer.json', local_dir = os.getcwd())

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Required - Connect to the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Initiate the connection to the target system.</p>

In [ ]:
#Create Connection to Vantage

eng = create_context(host = host, username = username, password = password)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 1 - Optional - Load the model into the database</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>First, connect to the target system.  Next, use teradataml functions to load the models to a specified table.  This table must have enough room to store model; see the <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_Lake_VMware/Teradata-VantageTM-Bring-Your-Own-Model-User-Guide/Preparing-to-Use-BYOM/Creating-Model-Tables'>user guide</a> for more details.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note</b> this step is optional if the model has already been loaded.  The model files can be quite large, so this is typically a one-time operation.</p>

In [ ]:
# Load Models into Vantage - the additional_columns argument can be used to pass additional metadata
if False:
    # a) Embedding model
    save_byom(model_id = model_name, # must be unique in the models table
              model_file = os.path.join(os.getcwd(), 'onnx', model_file_name),
              table_name = 'embeddings_models',
              schema_name = 'ai_demos',
              additional_columns = {'Dimensions':number_dimensions_output})

    # b) Tokenizer
    save_byom(model_id = model_name, # must be unique in the models table
                  model_file = 'tokenizer.json',
                  schema_name = 'ai_demos',
                  table_name = 'embeddings_tokenizers')

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Inspect the source data</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use simple python methods to inspect the Amazon Reviews data.  This code creates a teradataml DataFrame, which represents the data in the database which could extend to millions or billions of rows.  Data is not moved, and users can perform common data management and analytics functions that will run at scale on the target system.</p>

In [ ]:
tdf_reviews = DataFrame('"demo"."amazon_reviews_25"')
tdf_reviews.sample(2)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 2 - Generate embeddings</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_Lake_VMware/Teradata-VantageTM-Bring-Your-Own-Model-User-Guide/BYOM-Functions/ONNXEmbeddings'>ONNXEmbeddings</a> function can be expressed in SQL or in python using the teradatagenai library.  Each method takes multiple arguments, primarily:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li><b>Input Table</b> Either a query, table, or view that returns the source content.  This function assumes the text field is named 'txt'.</li>
    <li><b>Model Table</b> Can be a table or view if the entity contains a single model.  Else, pass a query that returns the single row for the selected model</li>
    <li><b>Tokenizer Table</b> As above; a table or view if single model table, or a query selecting a single row</li>
</ol> 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>SQL version:</b></p>

In [ ]:
input_table = 'demo.amazon_reviews_25'

qry = f'''
SELECT *
FROM td_mldb.ONNXEmbeddings(
        ON (SELECT TOP 1 id, rev_text AS txt FROM {input_table}) AS InputTable
        ON (SELECT * FROM ai_demos.embeddings_models WHERE model_id = '{model_name}') AS ModelTable DIMENSION
        ON (SELECT model AS tokenizer FROM ai_demos.embeddings_tokenizers WHERE model_id = '{model_name}') AS TokenizerTable DIMENSION
        USING
            Accumulate('id', 'txt')
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('FLOAT32({number_dimensions_output})')
            OverwriteCachedModel('true')
    ) AS embedding_output;
'''

tdf = DataFrame.from_query(qry)
tdf

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python version</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For python developers, the <a href = 'https://docs.teradata.com/r/Lake-Analyze-Your-Data-with-ClearScape-AnalyticsTM/Teradata-Package-for-Generative-AI'>teradatagenai</a> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local CPU or GPU compute. In the case of systems that don't have GPU support, the ONNXEmbeddings function will be used.</p>

In [ ]:
llm_onnx = TeradataAI(
    api_type = 'onnx',
    model_schema_name = 'ai_demos',
    model_name = model_name,
    model_id = model_name,
    model_table_name = 'embeddings_models',
    tokenizer_schema_name = 'ai_demos',
    tokenizer_id = model_name,
    tokenizer_table_name = 'embeddings_tokenizers')

# Instantiate the TextAnalyticsAI class with the ONNX model.
obj = TextAnalyticsAI(llm=llm_onnx)

tdf_embeddings = obj.embeddings(data = tdf_reviews,
                                column = 'rev_text', 
                                accumulate = 'id',
                                output_format = f'FLOAT32({number_dimensions_output})',
                                model_output_tensor = 'SENTENCE_EMBEDDING')

tdf_embeddings.sample(2)

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Store the embeddings</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For improved performance, store the embeddings in a table.  In this example, a volatile table is used so cleanup is automatic.</p>

In [ ]:
input_table = 'demo.amazon_reviews_25'

qry = f'''
CREATE VOLATILE TABLE review_embeddings AS (
SELECT *
FROM td_mldb.ONNXEmbeddings(
        ON (SELECT id, rev_text AS txt FROM {input_table}) AS InputTable
        ON (SELECT * FROM ai_demos.embeddings_models WHERE model_id = '{model_name}') AS ModelTable DIMENSION
        ON (SELECT model AS tokenizer FROM ai_demos.embeddings_tokenizers WHERE model_id = '{model_name}') AS TokenizerTable DIMENSION
        USING
            Accumulate('id', 'txt')
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('FLOAT32({number_dimensions_output})')
            OverwriteCachedModel('true')
    ) AS embedding_output) WITH DATA
ON COMMIT PRESERVE ROWS;
'''

execute_sql(qry)

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 3 - Perform Vector Distance calculations</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics function <a href = 'https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Model-Training-Functions/TD_VectorDistance'>TD_VectorDistance</a> function will take a table of input Vectors and a table of reference vectors and returns a table that contains the distance between target-reference pairs.  Since this function scans every row and performs the distance calculation, it is resource-intensive and usually suited to a lower number of records.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This example will use an input query to perform the distance calculations:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Pass the input text to the ONNXEmbedings function</li>
    <li>Use the resulting embeddings as a dimension to the distance function</li>
    <li>Examples are provided in python and SQL</li>
    </ol>
    
<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Embed the input text</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Rewrite the query to use input text for embedding.  Show the result.</p>

In [ ]:
search_term = input('Please enter a search term (e.g. "poetry"): ')

qry = f'''
SELECT *
FROM td_mldb.ONNXEmbeddings(
        ON (SELECT 1 id, '{search_term}' AS txt) AS InputTable
        ON (SELECT * FROM ai_demos.embeddings_models WHERE model_id = '{model_name}') AS ModelTable DIMENSION
        ON (SELECT model AS tokenizer FROM ai_demos.embeddings_tokenizers WHERE model_id = '{model_name}') AS TokenizerTable DIMENSION
        USING
            Accumulate('id', 'txt')
            ModelOutputTensor('sentence_embedding')
            EnableMemoryCheck('false')
            OutputFormat('FLOAT32({number_dimensions_output})')
            OverwriteCachedModel('true')
    ) AS embedding_output;
'''

tdf_input = DataFrame.from_query(qry)
print('Vector Representation of the input search term:')
ipydisplay(tdf_input.to_pandas())

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Perform Vector Distance calculation</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the volatile review embeddings table as the Reference table, and the question above as the Target table.</p>

In [ ]:
reference_table = 'review_embeddings'

dist_qry = f'''
SELECT TOP 10 target_id, reference_id, distancetype, ABS(cast(distance as decimal(36,8))) as distance FROM TD_VECTORDISTANCE (
    ON ({qry[:-2]}) AS TargetTable
    ON {reference_table} AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('[2:385]')
    RefIDColumn('id')
    RefFeatureColumns('[2:385]')
    DistanceMeasure('cosine')
    topk(10)
) AS dt order by 4;
'''
tdf_distance = DataFrame.from_query(dist_qry)
tdf_distance.to_pandas()

<hr>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Join the results back to the original complaints and topics</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this demonstration, expand all SQL to show the full query.</p>

In [ ]:
search_term = input('Please enter a search term (e.g "philosophy"): ')

join_qry = f'''
SELECT c.id review_id, c.rev_text review, d.distance

FROM (SELECT target_id, reference_id, distancetype, ABS(cast(distance as decimal(36,8))) as distance FROM TD_VECTORDISTANCE ( 
    ON 
        (SELECT *
        FROM td_mldb.ONNXEmbeddings(
                ON (SELECT 1 id, '{search_term}' AS txt) AS InputTable
                ON (SELECT * FROM ai_demos.embeddings_models WHERE model_id = '{model_name}') AS ModelTable DIMENSION
                ON (SELECT model AS tokenizer FROM ai_demos.embeddings_tokenizers WHERE model_id = '{model_name}') AS TokenizerTable DIMENSION
                USING
                    Accumulate('id', 'txt')
                    ModelOutputTensor('sentence_embedding')
                    EnableMemoryCheck('false')
                    OutputFormat('FLOAT32({number_dimensions_output})')
                    OverwriteCachedModel('true')
            ) AS embedding_output
    ) AS TargetTable
    ON {reference_table} AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('[2:385]')
    RefIDColumn('id')
    RefFeatureColumns('[2:385]')
    DistanceMeasure('cosine')
    topk(10)
) AS dt) d
    
JOIN {input_table} c
    ON c.id = d.reference_id;
'''
tdf_distance = DataFrame.from_query(join_qry)
tdf_distance.to_pandas()

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Python Version</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Perform the following operations using python classes and methods:</p>

<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Perform vector embedding of the query text</li>
    <li>Execute Vector Distance function</li>
    <li>Assemble final data set</li>
    </ol>


In [ ]:
search_term = input('Please enter a search term (e.g. "poetry"): ')

# re-initialize the byom object
llm_onnx = TeradataAI(
    api_type = 'onnx',
    model_schema_name = 'ai_demos',
    model_name = model_name,
    model_id = model_name,
    model_table_name = 'embeddings_models',
    tokenizer_schema_name = 'ai_demos',
    tokenizer_id = model_name,
    tokenizer_table_name = 'embeddings_tokenizers')

# Instantiate the TextAnalyticsAI class with the ONNX model.
obj = TextAnalyticsAI(llm=llm_onnx)

# generate an embedded representation of the search term
tdf_search_embedding = obj.embeddings(data = DataFrame.from_query(f'''SELECT 1 id, '{search_term}' AS txt'''),
                                column = 'txt', 
                                accumulate = 'id',
                                output_format = f'FLOAT32({number_dimensions_output})',
                                model_output_tensor = 'SENTENCE_EMBEDDING')
tdf_search_embedding 

In [ ]:
reference_table = 'review_embeddings'

tdf_dist = VectorDistance(target_data = tdf_search_embedding,
                          reference_data = DataFrame(reference_table),
                          target_id_column = 'id',
                          ref_id_column = 'id',
                          target_feature_columns = [f'emb_{str(i)}' for i in range(384)],
                          ref_feature_columns = [f'emb_{str(i)}' for i in range(384)],
                          topk = 10).result
                          
tdf_dist                     

<hr>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Step 3a - KMeans for fast search</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The native KMeans function can be used to create an Inverted File Index, where each cluster is an index.</p>

<table style = 'width:100%;table-layout:fixed;font-family:Arial;color:#00233C'>
    <tr><td style = 'vertical-align:top' width = '40%'>   
        <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Partitions data into clusters (Voronoi cells)</li>
    <li>Requires training to establish centroids (indexes)</li>
    <li>Using Teradata MPP architecture<ul><li>Distance computation from centroids is parallelized</li>
        <li>Distance computation from data points of a centroid is parallelized</li></ul></li>
            <li>Approximate approach. Trades off accuracy with speed. Query vectors near the boundary of a cluster may have nearest neighbor in the neighboring cluster</li>
    </ul>
</td><td style = 'text-align:center'><img src = 'images/KMEANS.png' width = '300'></td></tr>
</table>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Steps in the process:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Train the KMeans model - choosing an appropriate number of clusters for the volume of data.  This function returns two objects<ul><li>Model Table which contains cluster ID and centroid values</li><li>Cluster Assigments - the assigned cluster ID for each row in the source table</li></ul></li>
    <li>Use the Model Table as an index</li>
    <li>Execute Vector Distance on the index, returning N closest centroids</li>
    <li>Execute Vector distance on the original data, using the cluster assignments to filter the original data set</li>
</ol>
    
<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Train the KMeans model</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use SQL or Python interfaces to execute the <a href = 'https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Database-Analytic-Functions/Model-Training-Functions/TD_KMeans'>TD_KMEANS</a> function.  In this case, generate the model (with centroid values) AND cluster assignments:</p>

In [ ]:
try:
    execute_sql('DROP TABLE kmeans_model;')
    execute_sql('DROP TABLE cluster_assignments;')
except Exception as e:
    if 'does not exist.' in str(e):
        pass

# use the embedding table created above
input_table = 'review_embeddings'


qry = f'''
CREATE VOLATILE TABLE cluster_assignments AS (
SELECT * FROM TD_KMeans (
    ON {input_table} AS InputTable
    OUT VOLATILE TABLE ModelTable(kmeans_model)
USING
    IdColumn('id')
    TargetColumns('[2:385]')
    StopThreshold(0.0395)
    NumClusters(4)
    MaxIterNum(3)
    OutputClusterAssignment('true')
)AS dt) WITH DATA
ON COMMIT PRESERVE ROWS;
'''
execute_sql(qry)

ipydisplay(Markdown('Model: '))
ipydisplay(DataFrame('kmeans_model').to_pandas())

ipydisplay(Markdown('Cluster Assignments: '))
ipydisplay(DataFrame('cluster_assignments').to_pandas().head(5))

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Create Index table</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Use the model table as input to the Inverted File Index</p>

In [ ]:
try:
    execute_sql('DROP TABLE kmeans_index;')
except Exception as e:
    if 'does not exist.' in str(e):
        pass

qry = f'''
CREATE VOLATILE TABLE kmeans_index AS (
SELECT * FROM kmeans_model
WHERE td_clusterid_kmeans IS NOT NULL) WITH DATA
PRIMARY INDEX(td_clusterid_kmeans)
ON COMMIT PRESERVE ROWS;'''
execute_sql(qry)

ipydisplay(Markdown('Index Table: '))
ipydisplay(DataFrame('kmeans_index').to_pandas())

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Execute Vector Distance against index</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Return N (2 in this case) cluster assignments</p>

In [ ]:
# search_term = input('Please enter a search term: ')
search_term = 'Best books on philosophy'
reference_table = 'kmeans_index'

cluster_qry = f'''
SELECT reference_id cluster_id, distancetype, ABS(cast(distance as decimal(36,8))) as distance FROM TD_VECTORDISTANCE (
    ON 
        (SELECT *
        FROM td_mldb.ONNXEmbeddings(
                ON (SELECT 1 id, '{search_term}' AS txt) AS InputTable
                ON (SELECT * FROM ai_demos.embeddings_models WHERE model_id = '{model_name}') AS ModelTable DIMENSION
                ON (SELECT model AS tokenizer FROM ai_demos.embeddings_tokenizers WHERE model_id = '{model_name}') AS TokenizerTable DIMENSION
                USING
                    Accumulate('id', 'txt')
                    ModelOutputTensor('sentence_embedding')
                    EnableMemoryCheck('false')
                    OutputFormat('FLOAT32({number_dimensions_output})')
                    OverwriteCachedModel('true')
            ) AS embedding_output
    ) AS TargetTable
    ON {reference_table} AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('[2:385]')
    RefIDColumn('td_clusterid_kmeans')
    RefFeatureColumns('[1:384]')
    DistanceMeasure('cosine')
    topk(2)
) AS dt;
'''
tdf_distance = DataFrame.from_query(cluster_qry)
ipydisplay(Markdown(f'Search term: {search_term}'))
ipydisplay(tdf_distance.to_pandas())

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Use Cluster ID as a filter to Vector Distance</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Pass the top two closest cluster centroids as a filter to reduce the number of rows to scan with Vector Distance.  For this demo, use the filter explicitly.</p>

In [ ]:
# search_term = input('Please enter a search term: ')
search_term = 'Best books on philosophy'
input_table = 'demo.amazon_reviews_25'
reference_table = 'review_embeddings'
cluster_ids = '0,3'

join_qry = f'''
SELECT c.id review_id, c.rev_text review, d.distance

FROM (SELECT target_id, reference_id, distancetype, ABS(cast(distance as decimal(36,8))) as distance FROM TD_VECTORDISTANCE (
    ON 
        (SELECT *
        FROM td_mldb.ONNXEmbeddings(
                ON (SELECT 1 id, '{search_term}' AS txt) AS InputTable
                ON (SELECT * FROM ai_demos.embeddings_models WHERE model_id = '{model_name}') AS ModelTable DIMENSION
                ON (SELECT model AS tokenizer FROM ai_demos.embeddings_tokenizers WHERE model_id = '{model_name}') AS TokenizerTable DIMENSION
                USING
                    Accumulate('id', 'txt')
                    ModelOutputTensor('sentence_embedding')
                    EnableMemoryCheck('false')
                    OutputFormat('FLOAT32({number_dimensions_output})')
                    OverwriteCachedModel('true')
            ) AS embedding_output
    ) AS TargetTable
    ON 
        (SELECT * FROM {reference_table} 
        WHERE id IN (SELECT id FROM cluster_assignments WHERE td_clusterid_kmeans IN ({cluster_ids}))
    )  AS ReferenceTable DIMENSION
USING
    TargetIDColumn('id')
    TargetFeatureColumns('[2:385]')
    RefIDColumn('id')
    RefFeatureColumns('[2:385]')
    DistanceMeasure('cosine')
    topk(10)
) AS dt) d
    
JOIN {input_table} c
    ON c.id = d.reference_id;
'''
tdf_distance = DataFrame.from_query(join_qry)
ipydisplay(Markdown(f'Search term: {search_term}'))
ipydisplay(tdf_distance.to_pandas())

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Conclusion - Vector embedding and analytics - 17.20 and Vantage 3.0</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The preceding demo showed how users can generate vector embeddings <b>in-database</b> and then use native ClearScape Analytics functions to perform high-scale, parallelized similarity search</p>

<hr>
<p style = 'font-size:24px;font-family:Arial;color:#00233C'><b>Cleanup</b></p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Disconnect from the database to remove all volatile tables.</p>

In [ ]:
remove_context()